<a href="https://colab.research.google.com/github/otamajakusi/doc-vector/blob/main/sentence-transformers/bert_ldcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/otamajakusi/doc-vector

Cloning into 'doc-vector'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 13 (delta 3), reused 12 (delta 2), pack-reused 0
Unpacking objects: 100% (13/13), 4.56 KiB | 1.14 MiB/s, done.


In [3]:
%cd doc-vector/

/content/doc-vector


In [6]:
%cd sentence-transformers

/content/doc-vector/sentence-transformers


In [8]:
!bash download-ldcc.sh

--2023-04-22 01:08:17--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  1.71MB/s    in 4.9s    

2023-04-22 01:08:23 (1.71 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [10]:
!git pull

remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 1), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (4/4), 375 bytes | 375.00 KiB/s, done.
From https://github.com/otamajakusi/doc-vector
   12b0687..4c9772b  main       -> origin/main
Already up to date.


In [21]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.3/613.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 55.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=28ca7d24228f55c9f07c9

In [22]:
!python3 create_dataset.py

In [30]:
!python3 bert_classify.py

2023-04-22 01:39:22.361367: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using cuda device
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a Bert

In [32]:
!ls model_save

config.json	   special_tokens_map.json  vocab.txt
pytorch_model.bin  tokenizer_config.json


In [29]:
import tqdm.notebook

In [33]:
!ls -l model_save

total 432464
-rw-r--r-- 1 root root      1150 Apr 22 02:33 config.json
-rw-r--r-- 1 root root 442569845 Apr 22 02:33 pytorch_model.bin
-rw-r--r-- 1 root root       125 Apr 22 02:33 special_tokens_map.json
-rw-r--r-- 1 root root       497 Apr 22 02:33 tokenizer_config.json
-rw-r--r-- 1 root root    257706 Apr 22 02:33 vocab.txt


In [34]:
!pwd

/content/doc-vector/sentence-transformers


In [37]:
!mkdir /content/drive/MyDrive/ml-bert

In [38]:
!cp -r model_save /content/drive/MyDrive/ml-bert

In [39]:
!sync
